<center><img src="img/logo_hse_black.jpg"></center>

<h1><center>Natural Language Processing</center></h1>
<h2><center>Week1 Seminar - intro to TensorFlow</center></h2>

In [ ]:
%matplotlib inline

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

plt.style.use('ggplot')
plt.rcParams['figure.figsize'] = (12,8)

<center><img src='./img/tf.png'></center>

## TL;DR:
**TensorFlow** - библиотека для эффективного выполнения численных операций на графах вычислений 

Чтобы написать модель на TF надо

1. Сформировать граф вычислений (что на входе, что на выходе, как считать функцию потерь)
2. Запустить сессию по расчету графа (выделить память, подать на вход данные)
3. Немножно подождать..

Tensor - основной объект, над которыми производяться операции в TF. С математической точки зрения, тензор = многомерный массив
* 0-мерный тензор = число
* 1-мерный тензор = вектор
* 2-мерный тензор = матрица
* n-мерный тензор = .. тензор

С помощью команд, вы задаете операции над тензорами, результатом которых являются тоже тензоры

In [ ]:
import tensorflow as tf

In [ ]:
s1 = tf.Variable(2, dtype=tf.int32, name="scalar1") 
s2 = tf.Variable(3, name="scalar2") 
m = tf.Variable([[0, 1], [1, 2]], name="matrix") 
M = tf.Variable(tf.zeros([1000,8]))
M2 = tf.get_variable('another_matrix', 
                     dtype=tf.float32, 
                     initializer=tf.zeros([1000,8])) # Еще один, более совеременный вариант

In [ ]:
print(s1)

### Запускаем первый граф вычислений

In [ ]:
summ = s1+s2

In [ ]:
summ

`summ` - есмь граф вычислений. 

Надо запустить сессию, для его расчета

In [ ]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    print(sess.run(summ))

In [ ]:
summ.get_shape()

In [ ]:
M.get_shape()

Полный список математических операций можно найти [тут](https://www.tensorflow.org/api_docs/python/tf/math). А пока, давайте посмотрим на матричное уможение

In [ ]:
a = tf.constant([1, 2], name='a')
b = tf.constant([5, 2], name='b')
with tf.Session() as sess:
    print(sess.run(tf.multiply(a, b)))           
    print(sess.run(tf.tensordot(a, b, 1)))

In [ ]:
A = tf.constant([[1, 2], 
                 [3, 4]], 
                name='a')
with tf.Session() as sess:
    print(sess.run(tf.multiply(A, b)))           
    print(sess.run(tf.multiply(b, A)))           
    print(sess.run(tf.tensordot(A, b, 1)))
    print(sess.run(tf.tensordot(b, A, 1)))
    print(sess.run(tf.tensordot(A, A, 1)))

In [ ]:
A = tf.constant([[1, 2], 
                 [3, 4]], 
                name='a')
b = tf.constant([[5, 2]], name='b')
with tf.Session() as sess:
    print(sess.run(tf.matmul(b, A)))           
    print(sess.run(tf.matmul(A, b)))           

### Типы тензоров

* `tf.constant` - неизменяемые тензора, хранятся вместе с графом вычислений (плохо если они "тяжелые")
* `tf.Variable` - "переменные", обычно инициализируемые некоторым значением. Обычно язвляются обучаемыми переменными
* `tf.placeholder` - "заглушки". С их помощью мы можем заранее задать граф вычислений, но подавать данные уже после запуска сессии
* `tf.SparseTensor` - разреженный тензор

## С коробля на бал

Вспомним данные, которые у нас были [тут](https://nbviewer.jupyter.org/github/shestakoff/hse-ml-poly/blob/master/2018-10/s4-linear/seminar-linear-proc.ipynb)

In [ ]:
!wget https://www.dropbox.com/s/74c6mg219jnoa4k/accord_sedan_training.csv?dl=0 -O ./accord_sedan_training.csv    

In [ ]:
# Чтобы получать доступ к файлам, которые есть у вас на Google Drive
# from google.colab import drive
# drive.mount('/content/drive')


In [ ]:
df_train = pd.read_csv('./accord_sedan_training.csv')

In [ ]:
X_train = df_train.mileage.values.copy()
# X_train = (X_train - X_train.mean())/X_train.std()

y_train = df_train.price.values.copy()

Попробуем обучить модель вида 
`price = b + w*mileage`

Давайте прикинем типы "тензоров" данного графа вычислений


In [ ]:
tf.reset_default_graph()
y_input = ..
x_input = ..

b = ..
W = ..

In [ ]:
price = W*x_input+b

Пробуем вычислить price

In [ ]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    price_vals = sess.run(price, feed_dict = {x_input: X_train})

А теперь сделаем обучение линейной регрессии!

Challange - вспомнить и написать функцию потерь линейной регрессии с регуляризацией!

HINT: 
* `tf.reduce_mean`
* `tf.pow`
* `tf.squeeze`
* `tf.squared_difference`
* `tf.square`

In [ ]:
loss = tf.reduce_mean(tf.squared_difference(y_input,price))

После того, как вы задали loss, TF надо сказать каким методом ее оптимизировать, и всякие производные и обновления он сделает за вас!

Оптимизаторы можно найти в подмодуле `train` - обычно они заканчиваются на Optimizer

In [ ]:
optimizer = tf.train.AdamOptimizer(learning_rate=1.).minimize(loss)

Теперь вызовем сессию, которая будет исполнять граф вычислений для loss и optimizer 100 раз и выводить loss

In [ ]:
num_steps = 100000
batch_size = 64
n = X_train.shape[0]

with tf.Session() as sess:
    
    # Инициализируем переменные
    init = tf.global_variables_initializer()
    sess.run(init)
    
    # В цикле по количеству итераций
    for i in range(num_steps):
        # Выбираем индексы батча
        idx = np.random.randint(0, n, batch_size)
        
        # Запускаем сессии по расчету переменных
        _, loss_val, w_val, b_val = sess.run([optimizer, loss, W, b], 
                                                 feed_dict={x_input: X_train[idx], 
                                                            y_input: y_train[idx]})
        if i % 1000 == 0:
            print('Iteration {}: loss = {}'.format(i, loss_val))

    # Финальные предсказания модели
    y_hat = sess.run(price, feed_dict={x_input: X_train})

In [ ]:
w_val

In [ ]:
b_val

## Доп Материалы:
* [Доступный видос](https://www.youtube.com/watch?v=sEciSlAClL8) 
* [Курс лекций в Stanford](https://web.stanford.edu/class/cs20si/syllabus.html)